### Importamos las librerias que nos seran utiles

In [1017]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import seaborn as sns
import src.MetodosCleaningAnalysis as cf

#### Importamos los datos

In [1018]:
sharks = pd.read_csv('/home/julian/Cursos/Ironhack/Proyecto1/shark-data-cleaning/attacks.csv', encoding="ISO-8859-1")

In [1019]:
sdf = pd.DataFrame(sharks)

#### Como podemos observar, la mayoria de los datos son no numericos.

In [1020]:
sdf.dtypes

Case Number                object
Date                       object
Year                      float64
Type                       object
Country                    object
Area                       object
Location                   object
Activity                   object
Name                       object
Sex                        object
Age                        object
Injury                     object
Fatal (Y/N)                object
Time                       object
Species                    object
Investigator or Source     object
pdf                        object
href formula               object
href                       object
Case Number.1              object
Case Number.2              object
original order            float64
Unnamed: 22                object
Unnamed: 23                object
dtype: object

In [1021]:
sdf.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

## HIPOTESIS I

#### Vamos a eliminar las columnas que no son relevantes para el analisis de la primer hipotesis

In [1022]:
cf.keep_columns(['Case Number', 'Type', 'Case Number.1', 'Case Number.2'], sdf)

#### Las columnas 'Case Number', 'Case Number.1' y 'Case Number.2' son relevantes para el analisis, sin embargo, vamos a chequear en que medida los valores de las columnas se parecen para prescindir de alguna de las tres ##DEFINIR FUNCION

In [1023]:
"""
def compare(): 
        compare1 = np.where(sdf['Case Number'] == sdf['Case Number.1'], True, False)
        sdf['equal1'] = compare1
        compare2 = np.where(sdf['Case Number'] == sdf['Case Number.2'], True, False)
        sdf['equal2'] = compare2
        compare3 = np.where(sdf['Case Number'] == sdf['Case Number.2'], True, False)
        sdf['equal3'] = compare3
        return print(sdf['equal1'].value_counts()); print(sdf['equal2'].value_counts()); print(sdf['equal3'].value_counts())
"""
            

"\ndef compare(): \n        compare1 = np.where(sdf['Case Number'] == sdf['Case Number.1'], True, False)\n        sdf['equal1'] = compare1\n        compare2 = np.where(sdf['Case Number'] == sdf['Case Number.2'], True, False)\n        sdf['equal2'] = compare2\n        compare3 = np.where(sdf['Case Number'] == sdf['Case Number.2'], True, False)\n        sdf['equal3'] = compare3\n        return print(sdf['equal1'].value_counts()); print(sdf['equal2'].value_counts()); print(sdf['equal3'].value_counts())\n"

In [1024]:
compare = np.where(sdf['Case Number.1'] == sdf['Case Number.2'], True, False)

In [1025]:
sdf['equal'] = compare

In [1026]:
sdf['equal'].value_counts() 

False    19441
True      6282
Name: equal, dtype: int64

In [1027]:
sdf.isnull().sum()

Case Number      17021
Type             19425
Case Number.1    19421
Case Number.2    19421
equal                0
dtype: int64

#### SI COMPARAMOS LAS TRES COL ENTRE SI, EN DONDE MENOS DIF DE VAL HAY ES ENTRE CAS NBR Y CAS NBR.2, AUNQUE LA DI ES MINIMA. SIN EMBARGO DE LAS TRES COL LA QUE MENOS CANT DE NULL TIENE ES CASE NBR, NOS QUEDAMOS CON ELLA##DEJAMOS SOLO LAS COLUMNAS CASE NR Y TYPE

In [1028]:
cf.keep_columns(['Case Number', 'Type'], sdf)

#### VAMOS A ELIMINAR LOS VALORES QUE NO SEAN FECHA, YA QUE NOS INTERESA VER LA ESTACIONALIDAD. Empezamos por los valores NaN, y luego añadiremos una columna solo con los valores que contengan informacion sobre el mes, ya que nos interesa solo la indfo mensual

In [1029]:
sdf.dropna(inplace=True)

In [1030]:
sdf['Months'] = sdf['Case Number'].apply(cf.filtracion)

#### Borramos los valores nulos para los datos que no tenian registro mensual

In [1031]:
sdf.dropna(inplace=True)

#### exportamos el data frame a un csv para realizar las visualizaciones

In [1032]:
sdf.to_csv('sdf',index=False)

#### aHORA CONSTRUIMOS UN DATAFRAME CUYAS COLUMNAS CONTENDRAN LAS FRECUENCIAS MENSUALES DE ATAQUES Y LOS MESES CORRESPNDIENTES consideramos todos los tipos  de ataque.

In [1033]:
ataques_mensuales = pd.DataFrame(sdf['Months'].value_counts()).reset_index()

#### RENOMBRAMOS LAS COLUMNAS PARA EMPROLIJAR

In [1034]:
c = {'Months': 'Monthly Attacks', 'index':'Month'}
ataques_mensuales = ataques_mensuales.rename(columns=c)

In [1035]:
ataques_mensuales = ataques_mensuales.reindex([3, 11, 8, 5, 10, 4, 0, 1, 2, 7, 9, 6])

#### Exportamos el data frame de ataques mensuales para utilizarlo en las visualizaciones

In [1036]:
ataques_mensuales.to_csv('ataques_mensuales',index=False)

## HIPOTESIS II

In [1037]:
sdf2= pd.DataFrame(sharks)

### vamos a quedarnos solo con las columnas que nos interesan para analizar la H2, 'Species ' , 'Injury' y 'Fatal'

In [1038]:
cf.keep_columns(['Injury', 'Fatal (Y/N)', 'Species '], sdf2)

### vamos a limpiar las col 'Injury' y 'Fatal'

#### Si bien ambas columnas poseen datos sobre la letalidad, la columna 'Sepecies' es la que mas datos nulos posee. Comenzaremos limpiando esa columna

In [1039]:
sdf2['Fatal (Y/N)'].isna().value_counts()

True     19960
False     5763
Name: Fatal (Y/N), dtype: int64

In [1040]:
sdf2['Injury'].isna().value_counts()

True     19449
False     6274
Name: Injury, dtype: int64

In [1041]:
sdf2['Species '].isna().value_counts()

True     22259
False     3464
Name: Species , dtype: int64

In [1042]:
sdf2.dropna(inplace=True)

In [1043]:
sdf2['Identified Specie'] = sdf2['Species '].apply(cf.filtracion_especies)

### Vamos a armar dos data frames, uno con las muertes y otro que considere todos los ataques

### Ataques x especie

In [1044]:
ataques = pd.DataFrame(sdf2['Identified Specie'].value_counts()).reset_index()

In [1045]:
d = {'Identified Specie': 'Ataques', 'index':'Especie'}

In [1046]:
ataques = ataques.rename(columns=d)

#### Como hay muchas categorias de tiburones con tan solo un registro, vamos a considerar solo las que tengan al menos 10 registros de ataque.

In [1047]:
ataques = ataques[ataques['Ataques'] > 5]

In [1048]:
ataques.shape

(45, 2)

In [1049]:
ataques.to_csv('Ataques-especies',index=False)

### Letalidad por especie

#### Como la columna Fatal posee mas valores no nulos que Injury, potencialmente podemos obtener mas info sobre letalidad, vamos a prescindir de Injury. Vamos a quedarnos con los valores en donde hay un 'Y' en Fatal, en la col letalidad

In [1050]:
sdf2.drop(columns=['Injury'], inplace=True)

In [1051]:
letal = pd.DataFrame(sdf2[sdf2['Fatal (Y/N)'] == 'Y'])

In [1052]:
letal2 = pd.DataFrame(letal['Identified Specie'].value_counts()).reset_index()

In [1053]:
d = {'Identified Specie': 'Cantidad de Muertes', 'index':'Especie'}
letal2 = letal2.rename(columns=d)

#### Como podemos ver, a partir de la septima fila, los registros son muy, bajos, vamos a recortar el data frame para quedarnos con las especies que mas registros tienen

In [1054]:
letal2.drop(range(8, 41), axis=0, inplace=True)

In [1055]:
letal2.to_csv('especies-letalidad',index=False)

## HIPOTESIS III  Actividades/Ataques/Muertes

In [1056]:
sdf3 = pd.DataFrame(sharks)

#### Conservamos las columnas que nos interesan

In [1057]:
cf.keep_columns(['Activity', 'Fatal (Y/N)'], sdf3)

In [1058]:
sdf3['Fatal (Y/N)'].isna().value_counts()

True     19960
False     5763
Name: Fatal (Y/N), dtype: int64

In [1059]:
sdf3['Activity'].isna().value_counts()

True     19965
False     5758
Name: Activity, dtype: int64

#### Se puede observar que la cantidad de nulos es similar para ambas variables.

In [1060]:
sdf3.dropna(inplace=True)

#### Vamos a homogeneizar las columnas de actividades y fatalidades y eliminaremos los valores nulos

In [1061]:
sdf3['Actividades'] = sdf3['Activity'].apply(cf.filtracion_actividades)

In [1062]:
cf.keep_columns(['Actividades', 'Fatal (Y/N)'], sdf3)

In [1063]:
sdf3.dropna(inplace=True)

#### Definimos un df para registrar la cantidad de ataques no letales,  por actividad

In [1064]:
act = pd.DataFrame(sdf3[sdf3['Fatal (Y/N)'] == 'N'])

In [1065]:
act = pd.DataFrame(sdf3['Actividades'].value_counts()).reset_index()

In [1066]:
f = {'Actividades': 'Cantidad de Ataques', 'index':'Actividad'}

In [1067]:
act = act.rename(columns=f)

In [1068]:
act.head()

,Actividad,Cantidad de Ataques
0,surfing,1043
1,swimming,996
2,fishing,674
3,diving,471
4,spearfishing,379


#### Vamos a quedarnos con las actividades que registren mas de 20 ataques

In [1069]:
act = act[act['Cantidad de Ataques'] > 20]

In [1070]:
act.shape

(20, 2)

#### Definimos un df para registrar la cantidad de muertes por actividad

In [1071]:
act_let = pd.DataFrame(sdf3[sdf3['Fatal (Y/N)'] == 'Y'])

In [1072]:
act_let = pd.DataFrame(act_let['Actividades'].value_counts()).reset_index()

In [1073]:
e = {'Actividades': 'Cantidad de Muertes', 'index':'Actividad'}

In [1074]:
act_let = act_let.rename(columns=e)

In [1075]:
act_let.head()

,Actividad,Cantidad de Muertes
0,swimming,409
1,diving,121
2,fishing,96
3,bathing,84
4,surfing,60


#### Vamos a quedarnos con las actividades que registren mas de 20 muertes

In [1076]:
act_let = act_let[act_let['Cantidad de Muertes'] > 20]

In [1077]:
act_let.shape

(7, 2)

### vamos a unir ambos df

In [1078]:
#j = act.set_index('Actividad').join(act_let.set_index('Actividad')).reset_index()

In [1079]:
j = act.set_index('Actividad').join(act_let.set_index('Actividad'))

In [1080]:
j.dropna(inplace=True)

In [1081]:
j.to_csv('actividades-ataques.csv')